# `01-poi-enrich` - Adding Points of Interest from OSM

Import points of interest from the [Daylight Earth Table OpenStreetMap](https://daylightmap.org/earth/) distribution.

![](../img/data_model_pois.png)

In [12]:
# Bounding box to filter points of interest (San Mateo country)
bounds = {
    'lowerLeftLon': -122.562103,
    'lowerLeftLat': 37.425252,
    'upperRightLon': -122.211227,
    'upperRightLat': 37.701207,
}

# TODOs

- [x] add updated data model and arrows image
- [x] load daylight earth table poi into dataframe
- [x] iterate over dataframe to add pois to neo4j
- [x] bounding box to specify areas to include (think of a better way to do this)
- [x] find nearest intersection to attach poi node
- [ ] demonstrate routing between pois using apoc.algo.djikstra and apoc.algo.aStar
- [ ] document where to get data, or possibly use the API?


In [10]:
import pandas # TODO: poetry add pyarrow
from neo4j import GraphDatabase
import time
import json

## Load Daylight Earth Table Parquet Into Pandas DataFrame

In [11]:
# See data/README.md - the Daylight Earth Table is published as Parquet files in S3
# We will use Pandas to process these Parquet files and work with a DataFrame

df = pandas.read_parquet('../data/poi')

In [13]:
df

,geometry_id,class,subclass,metadata,original_source_tags,names,quadkey,wkt
0,n6124710136@2,commercial,fabric,"{""is_area"":false,""quadkey"":""120223012123112""}","{""addr:housenumber"":""11"",""addr:street"":""Corso ...","{""local"":""Tessilarte 81""}",120223012123112,POINT (7.5458496 44.3872338)
1,w919779614@1,parking,parking_space,"{""amenity"":""parking_space"",""is_area"":true,""qua...","{""amenity"":""parking_space"",""capacity"":""1"",""par...",{},031311121112031,POINT (-1.5464407275117145 54.91039713176086)
2,n289353412@2,recreation,playground,"{""is_area"":false,""quadkey"":""120221100321321""}","{""access"":""yes"",""leisure"":""playground""}",{},120221100321321,POINT (8.934112 48.527624)
3,n3241289585@2,tourism,viewpoint,"{""is_area"":false,""quadkey"":""030232112321013""}","{""tourism"":""viewpoint""}","{""local"":""Wanika Falls""}",030232112321013,POINT (-74.055717 44.1986092)
4,w407412726@1,commercial,retail,"{""building"":""retail"",""height_m"":5.5,""is_buildi...","{""building"":""retail"",""building:units"":""2"",""ele...",{},023012311310033,POINT (-118.2655498069126 33.98113907737265)
...,...,...,...,...,...,...,...,...
38777945,w375652631@1,parking,parking,"{""amenity"":""parking"",""is_area"":true,""quadkey"":...","{""amenity"":""parking"",""parking"":""surface""}",{},120223100310113,POINT (9.04611527653656 44.83077187234537)
38777946,w186376379@1,parking,parking,"{""amenity"":""parking"",""is_area"":true,""quadkey"":...","{""amenity"":""parking""}",{},121200312210212,POINT (49.41902012598777 53.50683910903766)
38777947,w73784956@2,recreation,swimming_pool,"{""is_area"":true,""quadkey"":""120222112310202""}","{""leisure"":""swimming_pool"",""source"":""cadastre-...",{},120222112310202,POINT (4.7539977549157975 44.29432166646483)
38777948,n4842464713@5,food,restaurant,"{""amenity"":""restaurant"",""is_area"":false,""quadk...","{""amenity"":""restaurant"",""check_date:opening_ho...","{""local"":""Zum Bären""}",120221120230310,POINT (8.6835075 47.5999337)


In [14]:
df.head()

,geometry_id,class,subclass,metadata,original_source_tags,names,quadkey,wkt
0,n6124710136@2,commercial,fabric,"{""is_area"":false,""quadkey"":""120223012123112""}","{""addr:housenumber"":""11"",""addr:street"":""Corso ...","{""local"":""Tessilarte 81""}",120223012123112,POINT (7.5458496 44.3872338)
1,w919779614@1,parking,parking_space,"{""amenity"":""parking_space"",""is_area"":true,""qua...","{""amenity"":""parking_space"",""capacity"":""1"",""par...",{},031311121112031,POINT (-1.5464407275117145 54.91039713176086)
2,n289353412@2,recreation,playground,"{""is_area"":false,""quadkey"":""120221100321321""}","{""access"":""yes"",""leisure"":""playground""}",{},120221100321321,POINT (8.934112 48.527624)
3,n3241289585@2,tourism,viewpoint,"{""is_area"":false,""quadkey"":""030232112321013""}","{""tourism"":""viewpoint""}","{""local"":""Wanika Falls""}",030232112321013,POINT (-74.055717 44.1986092)
4,w407412726@1,commercial,retail,"{""building"":""retail"",""height_m"":5.5,""is_buildi...","{""building"":""retail"",""building:units"":""2"",""ele...",{},023012311310033,POINT (-118.2655498069126 33.98113907737265)


## DataFrame Clean Up

In [15]:
# To start off let's filter for points of interest that have a name.
dffilter = df['names'].str.contains('local')
tempdf = df[dffilter]

In [16]:
# Next, let's convert some of the JSON strings stored in our DataFrame into Python dicts

def tryconvert(value, default):
    try:
        return json.loads(value)
    except Exception as e:
        pass
    return default

tempdf['names'] = tempdf['names'].apply(lambda x: tryconvert(x, {'name': "NA"}))

tempdf['original_tags_dict'] = tempdf['original_source_tags'].apply(lambda x: tryconvert(x, {}))


/var/folders/93/p_55x7950_32zryryx_jw6q00000gn/T/ipykernel_60765/2235055174.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf['names'] = tempdf['names'].apply(lambda x: tryconvert(x, {'name': "NA"}))
/var/folders/93/p_55x7950_32zryryx_jw6q00000gn/T/ipykernel_60765/2235055174.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf['original_tags_dict'] = tempdf['original_source_tags'].apply(lambda x: tryconvert(x, {}))


In [17]:
# And let's convert out Point geometry in WKT to a list of latitude/longitude coordinates
tempdf['point'] = tempdf['wkt'].apply(lambda x: x.replace('POINT (', "").replace(')', '').split())

/var/folders/93/p_55x7950_32zryryx_jw6q00000gn/T/ipykernel_60765/928612485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf['point'] = tempdf['wkt'].apply(lambda x: x.replace('POINT (', "").replace(')', '').split())


In [18]:
tempdf.head()

,geometry_id,class,subclass,metadata,original_source_tags,names,quadkey,wkt,original_tags_dict,point
0,n6124710136@2,commercial,fabric,"{""is_area"":false,""quadkey"":""120223012123112""}","{""addr:housenumber"":""11"",""addr:street"":""Corso ...",{'local': 'Tessilarte 81'},120223012123112,POINT (7.5458496 44.3872338),"{'addr:housenumber': '11', 'addr:street': 'Cor...","[7.5458496, 44.3872338]"
3,n3241289585@2,tourism,viewpoint,"{""is_area"":false,""quadkey"":""030232112321013""}","{""tourism"":""viewpoint""}",{'local': 'Wanika Falls'},030232112321013,POINT (-74.055717 44.1986092),{'tourism': 'viewpoint'},"[-74.055717, 44.1986092]"
6,n4740980225@1,other,poi,"{""is_area"":false,""quadkey"":""132221130201020""}","{""phone"":""+6677270670"",""tourism"":""hotel""}",{'local': 'Centra Coconut beach resort samui'},132221130201020,POINT (99.9374697 9.4192596),"{'phone': '+6677270670', 'tourism': 'hotel'}","[99.9374697, 9.4192596]"
9,n7106398577@1,medical,hospital,"{""amenity"":""hospital"",""is_area"":false,""quadkey...","{""addr:district"":""Ludhiana"",""addr:full"":""Line ...","{'local': 'Raman Hospital, Ludhiana'}",123121013312320,POINT (75.8115623 30.9155611),"{'addr:district': 'Ludhiana', 'addr:full': 'Li...","[75.8115623, 30.9155611]"
10,n1952358265@4,food,restaurant,"{""amenity"":""restaurant"",""is_area"":false,""quadk...","{""addr:city"":""Roma"",""addr:country"":""IT"",""addr:...",{'local': 'Pizzeria L'Economica'},120232221130010,POINT (12.5128048 41.8974497),"{'addr:city': 'Roma', 'addr:country': 'IT', 'a...","[12.5128048, 41.8974497]"


## Neo4j Import

See https://towardsdatascience.com/create-a-graph-database-in-neo4j-using-python-4172d40f89c4

In [19]:
# Let's create a class to abstract the Neo4j Python driver API

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exeception as e:
            print("Failed to create the driver:", e)
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
    
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            # TODO: convert to transaction function / 5.0 driver API
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

conn = Neo4jConnection(uri="neo4j://localhost", user="neo4j", pwd="letmeinnow")

In [20]:
# Here we define a function to split our DataFrame into batches for import

def insert_data(query, bounds, rows, batch_size=100000):
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):
        res = conn.query(query, parameters = {'bounds': bounds, 'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')}, db="neo4j")
        total += res[0]['total']
        batch += 1
        result = {"total": total, "batches": batch, "time": time.time()-start}
        print(result)
    return result

In [29]:
# We'll define a Cypher Query to create data in Neo4j

def insert_pois(rows, batch_size=10000):
    print(rows)
    
    query = '''
    UNWIND $rows AS row // TODO: insert bounding box for san mateo county
    WITH row WHERE point.withinBBox(
            point({longitude: toFloat(row.point[0]), latitude: toFloat(row.point[1])}), 
            point({longitude: $bounds.lowerLeftLon, latitude: $bounds.lowerLeftLat }), 
            point({longitude: $bounds.upperRightLon, latitude: $bounds.upperRightLat}))
    // TODO: MERGE on geometry_id, create constraint
    CREATE (p:PointOfInterest {name: row.names.local, geometry_id: row.geometry_id}) 
    SET p.location = point({latitude: toFloat(row.point[1]), longitude: toFloat(row.point[0])}),
        p.class = row.class,
        p.subclass = row.subclass
    CREATE (t:Tags)
    SET t += row.original_tags_dict
    CREATE (p)-[:HAS_TAGS]->(t)
    RETURN COUNT(*) AS total
    '''
    
    return insert_data(query, bounds, rows, batch_size,)

In [30]:
insert_pois(tempdf)

            geometry_id          class       subclass  \
0         n6124710136@2     commercial         fabric   
3         n3241289585@2        tourism      viewpoint   
6         n4740980225@1          other            poi   
9         n7106398577@1        medical       hospital   
10        n1952358265@4           food     restaurant   
...                 ...            ...            ...   
38777936  n2616349967@3        amenity           bank   
38777937   n260756960@3            bar            pub   
38777938  n4851360339@2         public        library   
38777940   w231023584@3  entertainment  sports_centre   
38777948  n4842464713@5           food     restaurant   

                                                   metadata  \
0             {"is_area":false,"quadkey":"120223012123112"}   
3             {"is_area":false,"quadkey":"030232112321013"}   
6             {"is_area":false,"quadkey":"132221130201020"}   
9         {"amenity":"hospital","is_area":false,"quadkey...   


{'total': 143, 'batches': 53, 'time': 43.745259046554565}
{'total': 145, 'batches': 54, 'time': 44.47097897529602}
{'total': 151, 'batches': 55, 'time': 45.29883694648743}
{'total': 157, 'batches': 56, 'time': 46.0220890045166}
{'total': 160, 'batches': 57, 'time': 46.8612060546875}
{'total': 164, 'batches': 58, 'time': 47.58425688743591}
{'total': 165, 'batches': 59, 'time': 48.40336298942566}
{'total': 168, 'batches': 60, 'time': 49.11611199378967}
{'total': 168, 'batches': 61, 'time': 49.915539026260376}
{'total': 170, 'batches': 62, 'time': 50.67403793334961}
{'total': 172, 'batches': 63, 'time': 51.581966161727905}
{'total': 176, 'batches': 64, 'time': 52.244426012039185}
{'total': 180, 'batches': 65, 'time': 53.071484088897705}
{'total': 183, 'batches': 66, 'time': 53.80159306526184}
{'total': 185, 'batches': 67, 'time': 54.65021109580994}
{'total': 188, 'batches': 68, 'time': 55.37458395957947}
{'total': 189, 'batches': 69, 'time': 56.20565700531006}
{'total': 189, 'batches': 70

{'total': 535, 'batches': 195, 'time': 154.38262701034546}
{'total': 538, 'batches': 196, 'time': 155.25372219085693}
{'total': 541, 'batches': 197, 'time': 155.95437502861023}
{'total': 546, 'batches': 198, 'time': 156.6394600868225}
{'total': 549, 'batches': 199, 'time': 157.48122596740723}
{'total': 552, 'batches': 200, 'time': 158.19363808631897}
{'total': 554, 'batches': 201, 'time': 158.880695104599}
{'total': 556, 'batches': 202, 'time': 159.72813510894775}
{'total': 558, 'batches': 203, 'time': 160.4398729801178}
{'total': 558, 'batches': 204, 'time': 161.13260793685913}
{'total': 561, 'batches': 205, 'time': 161.9866259098053}
{'total': 568, 'batches': 206, 'time': 162.71483993530273}
{'total': 570, 'batches': 207, 'time': 163.41645693778992}
{'total': 574, 'batches': 208, 'time': 164.24646496772766}
{'total': 577, 'batches': 209, 'time': 164.953143119812}
{'total': 579, 'batches': 210, 'time': 165.66044807434082}
{'total': 583, 'batches': 211, 'time': 166.50307703018188}
{'to

{'total': 866, 'batches': 335, 'time': 265.1347761154175}
{'total': 868, 'batches': 336, 'time': 265.9581491947174}
{'total': 868, 'batches': 337, 'time': 266.65170097351074}
{'total': 869, 'batches': 338, 'time': 267.3518841266632}
{'total': 874, 'batches': 339, 'time': 268.17775297164917}
{'total': 877, 'batches': 340, 'time': 268.90423607826233}
{'total': 880, 'batches': 341, 'time': 269.6008770465851}
{'total': 883, 'batches': 342, 'time': 270.4219779968262}
{'total': 887, 'batches': 343, 'time': 271.1267068386078}
{'total': 889, 'batches': 344, 'time': 272.0100588798523}
{'total': 891, 'batches': 345, 'time': 272.6979949474335}
{'total': 893, 'batches': 346, 'time': 273.54612398147583}
{'total': 898, 'batches': 347, 'time': 274.27507615089417}
{'total': 900, 'batches': 348, 'time': 275.1156520843506}
{'total': 900, 'batches': 349, 'time': 275.77365803718567}
{'total': 907, 'batches': 350, 'time': 276.6404941082001}
{'total': 908, 'batches': 351, 'time': 277.38480401039124}
{'total

{'total': 1229, 'batches': 474, 'time': 375.0048339366913}
{'total': 1229, 'batches': 475, 'time': 375.6926009654999}
{'total': 1231, 'batches': 476, 'time': 376.52993416786194}
{'total': 1234, 'batches': 477, 'time': 377.25578904151917}
{'total': 1237, 'batches': 478, 'time': 378.1049120426178}
{'total': 1239, 'batches': 479, 'time': 378.82116198539734}
{'total': 1239, 'batches': 480, 'time': 379.67225217819214}
{'total': 1243, 'batches': 481, 'time': 380.55420207977295}
{'total': 1244, 'batches': 482, 'time': 381.3035628795624}
{'total': 1247, 'batches': 483, 'time': 382.014799118042}
{'total': 1251, 'batches': 484, 'time': 382.81845021247864}
{'total': 1252, 'batches': 485, 'time': 383.52240204811096}
{'total': 1256, 'batches': 486, 'time': 384.3251609802246}
{'total': 1257, 'batches': 487, 'time': 385.23864698410034}
{'total': 1260, 'batches': 488, 'time': 386.03133893013}
{'total': 1263, 'batches': 489, 'time': 386.84070014953613}
{'total': 1263, 'batches': 490, 'time': 387.611981

...

{'total': 2981, 'batches': 1169, 'time': 918.6611278057098}
{'total': 2984, 'batches': 1170, 'time': 919.4130940437317}
{'total': 2986, 'batches': 1171, 'time': 920.1143200397491}
{'total': 2990, 'batches': 1172, 'time': 920.9370319843292}
{'total': 2991, 'batches': 1173, 'time': 921.6509301662445}
{'total': 2992, 'batches': 1174, 'time': 922.366895198822}
{'total': 2995, 'batches': 1175, 'time': 923.2094841003418}
{'total': 2995, 'batches': 1176, 'time': 923.9195189476013}
{'total': 2999, 'batches': 1177, 'time': 924.6036882400513}
{'total': 3003, 'batches': 1178, 'time': 925.4379901885986}
{'total': 3005, 'batches': 1179, 'time': 926.1405041217804}
{'total': 3009, 'batches': 1180, 'time': 926.8353500366211}
{'total': 3015, 'batches': 1181, 'time': 927.6489789485931}
{'total': 3018, 'batches': 1182, 'time': 928.3625519275665}
{'total': 3018, 'batches': 1183, 'time': 929.0415511131287}
{'total': 3024, 'batches': 1184, 'time': 929.8866119384766}
{'total': 3026, 'batches': 1185, 'time': 

{'total': 3366, 'batches': 1306, 'time': 1024.6301510334015}
{'total': 3369, 'batches': 1307, 'time': 1025.320796251297}
{'total': 3371, 'batches': 1308, 'time': 1026.2146220207214}
{'total': 3375, 'batches': 1309, 'time': 1026.882553100586}
{'total': 3379, 'batches': 1310, 'time': 1027.5823979377747}
{'total': 3384, 'batches': 1311, 'time': 1028.4259130954742}
{'total': 3387, 'batches': 1312, 'time': 1029.1341650485992}
{'total': 3389, 'batches': 1313, 'time': 1029.8294241428375}
{'total': 3392, 'batches': 1314, 'time': 1030.638789176941}
{'total': 3397, 'batches': 1315, 'time': 1031.359803199768}
{'total': 3400, 'batches': 1316, 'time': 1032.0196678638458}
{'total': 3401, 'batches': 1317, 'time': 1032.8315160274506}
{'total': 3405, 'batches': 1318, 'time': 1033.545835018158}
{'total': 3405, 'batches': 1319, 'time': 1034.22780418396}
{'total': 3405, 'batches': 1320, 'time': 1034.995227098465}
{'total': 3407, 'batches': 1321, 'time': 1035.7191591262817}
{'total': 3408, 'batches': 1322,

{'total': 3748, 'batches': 1442, 'time': 1128.4383358955383}
{'total': 3751, 'batches': 1443, 'time': 1129.2253019809723}
{'total': 3752, 'batches': 1444, 'time': 1129.9650540351868}
{'total': 3755, 'batches': 1445, 'time': 1130.7779870033264}
{'total': 3761, 'batches': 1446, 'time': 1131.51766705513}
{'total': 3764, 'batches': 1447, 'time': 1132.1952481269836}
{'total': 3766, 'batches': 1448, 'time': 1133.0222730636597}
{'total': 3768, 'batches': 1449, 'time': 1133.7400901317596}
{'total': 3773, 'batches': 1450, 'time': 1134.498857975006}
{'total': 3774, 'batches': 1451, 'time': 1135.460464000702}
{'total': 3780, 'batches': 1452, 'time': 1136.1496131420135}
{'total': 3781, 'batches': 1453, 'time': 1136.8403761386871}
{'total': 3788, 'batches': 1454, 'time': 1137.5412459373474}
{'total': 3790, 'batches': 1455, 'time': 1138.3066852092743}
{'total': 3792, 'batches': 1456, 'time': 1142.8729121685028}
{'total': 3798, 'batches': 1457, 'time': 1143.6886839866638}
{'total': 3802, 'batches': 1

{'total': 4138, 'batches': 1577, 'time': 1236.449877023697}
{'total': 4138, 'batches': 1578, 'time': 1237.1604959964752}
{'total': 4140, 'batches': 1579, 'time': 1238.0130870342255}
{'total': 4143, 'batches': 1580, 'time': 1238.8878920078278}
{'total': 4145, 'batches': 1581, 'time': 1239.7450859546661}
{'total': 4147, 'batches': 1582, 'time': 1240.4915380477905}
{'total': 4151, 'batches': 1583, 'time': 1241.318459033966}
{'total': 4156, 'batches': 1584, 'time': 1242.0532898902893}
{'total': 4158, 'batches': 1585, 'time': 1242.844237089157}
{'total': 4158, 'batches': 1586, 'time': 1243.5694830417633}
{'total': 4160, 'batches': 1587, 'time': 1244.3984830379486}
{'total': 4161, 'batches': 1588, 'time': 1245.1099262237549}
{'total': 4166, 'batches': 1589, 'time': 1245.9857881069183}
{'total': 4170, 'batches': 1590, 'time': 1246.7463221549988}
{'total': 4173, 'batches': 1591, 'time': 1247.5104339122772}
{'total': 4175, 'batches': 1592, 'time': 1248.28076004982}
{'total': 4176, 'batches': 15

{'total': 4319, 'batches': 1653, 'time': 1294.4179770946503}

Create a POINT index on Intersection.location:

```Cypher
CREATE POINT INDEX FOR (i:Intersection) ON i.location
```


Attach each `PointOfInterest` node to the near `Intersection` node:

```Cypher
MATCH (p:PointOfInterest) WHERE NOT EXISTS ((p)-[:NEAREST_INTERSECTION]->(:Intersection))
WITH p LIMIT 1000
CALL {
  WITH p
  MATCH (i:Intersection)
  USING INDEX i:Intersection(location)
  WHERE point.distance(i.location, p.location) < 2000
  WITH i
  ORDER BY point.distance(p.location, i.location) ASC 
  LIMIT 1
  RETURN i
}
WITH p, i

MERGE (p)-[r:NEAREST_INTERSECTION]->(i)
SET r.length = point.distance(p.location, i.location)
RETURN COUNT(p)
```

In [ ]:
# TODO: routing